<a href="https://colab.research.google.com/github/bmurphy2019/groupProjectAi/blob/main/AlzheimersAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.callbacks import ModelCheckpoint


# data importing
# I picked oasis cross sectional since its the bigger of the two data sets
data_cross = pd.read_csv("/content/oasis_cross-sectional.csv")

data_cross.columns = ['ID','Gender','Hand','Age','Educ','SES','MMSE','CDR','eTIV','nWBV','ASF','Delay']
# data cleaning
data_cross.dropna(subset=['CDR'],inplace=True)

data_cross.drop(columns=['ID','Delay'],inplace=True)

data = data_cross

# Dropping all the rows with nan values 
data.dropna(inplace=True) 

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data['CDR'] = le.fit_transform(data['CDR'].values)
data = data.drop(data[data['CDR']==3].index)

data['Gender'] = data.Gender.map({'M': 0, 'F': 1})
data['CDR'] = data.CDR.map({0: 0, 1: 1, 2:1})
data['Hand'] = data.Hand.map({'R': 0, 'L': 1})
# add target column to Y and everything else to X
y = data.pop('CDR')
X = data

# 80-20 split train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)
model = Sequential()
model.add(Dense(30, activation = 'relu', input_dim=9))

model.add(Dense(20, activation='relu'))

model.add(Dense(15, activation='relu')) 

model.add(Dense(1, activation='sigmoid')) 

model.compile(optimizer= 'adam' ,loss='binary_crossentropy',metrics=['acc'])




checkpointer = ModelCheckpoint('Alzheimers.h5', monitor='val_acc', mode='max', verbose=2, save_best_only=True)
history=model.fit(X_train, y_train, batch_size=16, epochs=350, validation_data=(X_test, y_test), callbacks=[checkpointer])



Epoch 1/350
 1/11 [=>............................] - ETA: 5s - loss: 80.8367 - acc: 0.6250
Epoch 00001: val_acc improved from -inf to 0.32558, saving model to Alzheimers.h5
11/11 [==============================] - 1s 24ms/step - loss: 43.4267 - acc: 0.5789 - val_loss: 11.7016 - val_acc: 0.3256
Epoch 2/350
 1/11 [=>............................] - ETA: 0s - loss: 10.5815 - acc: 0.3750
Epoch 00002: val_acc improved from 0.32558 to 0.67442, saving model to Alzheimers.h5
11/11 [==============================] - 0s 9ms/step - loss: 9.9801 - acc: 0.3918 - val_loss: 6.8955 - val_acc: 0.6744
Epoch 3/350
 1/11 [=>............................] - ETA: 0s - loss: 5.5303 - acc: 0.7500
Epoch 00003: val_acc did not improve from 0.67442
11/11 [==============================] - 0s 5ms/step - loss: 4.0964 - acc: 0.5497 - val_loss: 1.8367 - val_acc: 0.6744
Epoch 4/350
 1/11 [=>............................] - ETA: 0s - loss: 2.1742 - acc: 0.6250
Epoch 00004: val_acc did not improve from 0.67442
11/11 [====

In [6]:
# simple accuracy evauation and print
present_model = keras.models.load_model('Alzheimers.h5')
print("Accuracy of our model on test data : " , present_model.evaluate(X_test,y_test)[1]*100 , "%")

2/2 [==============================] - 0s 6ms/step - loss: 0.5486 - acc: 0.8140
Accuracy of our model on test data :  81.39534592628479 %


In [7]:
# export as tflite model
import tensorflow as tf

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(keras.models.load_model('Alzheimers.h5'))
tflite_model = converter.convert()

# Save the model.
with open('Alzheimers.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpljaubyvo/assets
